In [2]:
from netgen.meshing import *
from netgen.csg import *
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
import math

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

BaseWebGuiScene

In [28]:
a=0.01
b=0.1
h=0.05
ndiv=36
conductor = Cylinder((0,0,0), Z, r=a, h=h)
conductor.maxh=2*math.pi*a/ndiv
#Draw(conductor)
conductor.faces.name="conductorBND"
conductor.faces.Max(Z).name="in"
conductor.faces.Min(Z).name="out"
conductor.mat("sig")
#crosssection = conductor.faces.Max(Z).mass
#print(crosssection)

outer = Cylinder((0,0,0), Z, r=b, h=h)
outer.faces.name = "outer"
air = outer-conductor
air.maxh=2*math.pi*b/ndiv
air.faces.Max(Z).name="upper"
air.faces.Min(Z).name="lower"
air.mat("air")
model=Glue([conductor,air])

Draw(model)
geo =OCCGeometry(model)
with TaskManager():
    mesh = Mesh(geo.GenerateMesh()).Curve(1) 
Draw(mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

0.0003141592653589792


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [37]:
fesA = HCurl(mesh, order=1, nograds=True,  dirichlet="out|in|upper|lower", complex=True)
A,N = fesA.TnT()
mu = 4*math.pi*1e-7
sigma = mesh.MaterialCF({ "sig" : 1e6 }, default=0)
#sigma=1.e6
freq=1.
outerRadius=b
current=1.
h=current/(2*math.pi*outerRadius)
s=1j*2*math.pi*freq
a=BilinearForm(fesA)
a += 1/mu*curl(A)*curl(N)*dx
a += 1e-6/mu*A*N*dx
a += sigma*s*A*N*dx("sig")
f = LinearForm(fesA)
f += -N.Trace()*(0,0,1)*h*ds("outer")

gfA = GridFunction(fesA)

with TaskManager():
    c = Preconditioner(a, type="bddc")
    a.Assemble()
    f.Assemble()

In [38]:
solvers.CG(sol=gfA.vec, rhs=f.vec, mat=a.mat, pre=c.mat, tol=1e-9, printrates=True, maxsteps=1000)

CG iteration 1, residual = 0.005034266514168248     
CG iteration 2, residual = 1.2178366204936044e-09     
CG iteration 3, residual = 9.777786172062313e-12     
CG iteration 4, residual = 1.144014410586086e-14     


basevector

In [41]:
Draw(sigma, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [31]:
Draw (curl(gfA), mesh, draw_surf=False, \
      min=0, max=1.e-5, clipping = { "y":1, "z" : 0, "function":False}, vectors = { "grid_size":100});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…